# Looking into iGraph and Les Mis data

The [Cho Energy PPT](http://web.ecs.baylor.edu/faculty/cho/Cho-Entropy.pdf)  He has the entropy calculation as

$$e(v) = -p_i \log(p_i) - p_o\log(p_o)$$

where $i$ is the internal weight and $o$ is the external weight.  One thing I keep forgetting is that you take the entropy of the whole graph, not just the sub-graph

[Here](https://blog.thedataincubator.com/2015/08/embedding-d3-in-an-ipython-notebook/) is a reference for javascript in Jupyter. No, use http://www.primerpy.com/post/js/d3-visualization-embedded-in-jupyter-notebook/ 

In [258]:
from IPython.core.display import display, HTML
from string import Template
import igraph
import json
import math
import copy

In [259]:
HTML('<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.16/d3.min.js"></script>')

In [260]:
css_text = '''
'''

In [261]:
js_text_template = Template('''
       var bogoSVG = d3.select("#animation") 
          .append("svg")
          .attr("width", 300)
          .attr("height", 300);    

      var data = $python_data ;
       bogoSVG.append("circle")
          .style("stroke", "gray")
          .style("fill", "cyan")
          .attr("r", data[0]['r'])
          .attr("cx", data[0]['cx'])
          .attr("cy", data[0]['cy'])
          .transition()
             .delay(100)
             .duration(8000)  
             .attr("r", 10)
             .attr("cx", data[0]['cx'])
             .style("fill", "blue"); 
''')

In [262]:
html_template = Template('''
<style> $css_text </style>
<div id="animation"></div>
<script> $js_text </script>
''')

In [263]:
js_text = js_text_template.substitute({'python_data': json.dumps([{'r': 130, 'cx': 150, 'cy': 150}])})

In [264]:
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))
# Refresh the page to see the animation again

# The Graph

In [166]:
#fn = "data/miserables.json"
fn = "data/guardians.json"
with open(fn, 'r') as f:
    mis = json.load(f)

In [167]:
g = igraph.Graph()

In [168]:
for n in mis['nodes']:
    g.add_vertex(n['name'])
for l in mis['links']:
    g.add_edge(l['source'], l['target'], weight=l['value'])

In [243]:
def vertex_flow(subgraph, v):
    internal_energy = 0
    external_energy = 0
    nbs = v.neighbors()
    nids = [x.index for x in subgraph]
    v_base_energy = 0.0
    for nb in nbs:
        v_base_energy += sum([e['weight'] for e in g.es.select(_source=v.index,_target=nb.index)])
        v_base_energy += sum([e['weight'] for e in g.es.select(_source=nb.index,_target=v.index)])

    for nb in nbs:
        w = sum([e['weight'] for e in g.es.select(_source=v.index,_target=nb.index)]) / v_base_energy
        w += sum([e['weight'] for e in g.es.select(_source=nb.index,_target=v.index)]) / v_base_energy
        if nb.index in nids:
            internal_energy += w
        else:
            external_energy += w
 
    p = internal_energy/(internal_energy + external_energy)

    e = 0
    if p >0:
        e += -p*math.log(p,2)
    if p < 1:
        e+= -(1-p) * math.log(1-p,2)
    return e

In [252]:
subgraph = g.vs([0,1,2,3])
vertex_flow(subgraph, g.vs[3])

0.8112781244591328

In [255]:
vertex_flow(subgraph, g.vs[5])

0.0

In [249]:
def boundary_entropy(subgraph):
    internal_energy = 0
    external_energy = 0
    base_energy = 0
    nids = [x.index for x in subgraph]

    entropy = 0
    for v in subgraph:
        entropy += vertex_flow(subgraph, v)
    
    return entropy

In [256]:
subgraph = g.vs([0,1,2,3])
boundary_entropy(subgraph)

0.8112781244591328

In [142]:
def prune_boundary(subgraph, current_entropy):
    resultant_subgraph = copy.copy(subgraph)
    
    for v in subgraph:
        sbg = copy.copy(subgraph)
        sbg.remove(v)
        sbg_entropy = boundary_entropy(sbg)
        if sbg_entropy < current_entropy:
            resultant_subgraph.remove(v)
    resultant_entropy = boundary_entropy(resultant_subgraph)
    return (resultant_subgraph, resultant_entropy)

In [139]:
def grow_boundary(subgraph, current_entropy):
    return 0

In [150]:
def print_boundary(subgraph):
    msg = ''
    for v in subgraph:
        msg += ' %s ' % v['name']
    print(msg)
    return msg

In [180]:
def run():
    for v in g.vs():
        # First order neighbors
        nbs = v.neighbors()
        nbs.append(v)  # Need to be in your own set
        interior = copy.copy(nbs)
        nids = [x.index for x in nbs]
        # Begin with the first order neigbors of v
        entropy = boundary_entropy(nbs)      
        # Start pruning
        print("\n ** %s len(interior)  (pre): %s  %s"  % (v['name'], len(interior), entropy))
        print_boundary(interior)
        interior, entropy = prune_boundary(interior, entropy)
        print_boundary(interior)
        print("    %s len(interior) (post): %s  %s" % (v['name'], len(interior), entropy))



In [185]:
run()

Gamora v_base_energy: 2.0
Drax v_base_energy: 2.0
Rocket v_base_energy: 4.0
Star Lord v_base_energy: 3.0

 ** Star Lord len(interior)  (pre): 4  3.87120101091
 Gamora  Drax  Rocket  Star Lord 
Drax v_base_energy: 2.0
Rocket v_base_energy: 4.0
Star Lord v_base_energy: 3.0
Gamora v_base_energy: 2.0
Rocket v_base_energy: 4.0
Star Lord v_base_energy: 3.0
Gamora v_base_energy: 2.0
Drax v_base_energy: 2.0
Star Lord v_base_energy: 3.0
Gamora v_base_energy: 2.0
Drax v_base_energy: 2.0
Rocket v_base_energy: 4.0

    Star Lord len(interior) (post): 0  0
Star Lord v_base_energy: 3.0
Rocket v_base_energy: 4.0
Gamora v_base_energy: 2.0

 ** Gamora len(interior)  (pre): 3  3.17805383035
 Star Lord  Rocket  Gamora 
Rocket v_base_energy: 4.0
Gamora v_base_energy: 2.0
Star Lord v_base_energy: 3.0
Gamora v_base_energy: 2.0
Star Lord v_base_energy: 3.0
Rocket v_base_energy: 4.0

    Gamora len(interior) (post): 0  0
Star Lord v_base_energy: 3.0
Rocket v_base_energy: 4.0
Drax v_base_energy: 2.0

 ** Drax 

In [60]:
sum([x['weight'] for x in g.es.select(_source=0, _target=1)])

1

In [173]:
print(g)

IGRAPH UNW- 8 10 --
+ attr: name (v), weight (e)
+ edges (vertex names):
Star Lord--Gamora, Star Lord--Drax, Star Lord--Rocket, Gamora--Rocket,
Drax--Rocket, Rocket--Groot, Groot--Drax, Drax--Yondu, Drax--Nebula,
Yondu--Nebula


In [174]:
sl = g.vs[0]

In [176]:
nbs = sl.neighbors()

In [178]:
nbs.append(sl)